# 🇷🇺 Russia Fuel Crisis: Analyse Interactive (Modèle Hybride)

Ce notebook permet de **reproduire** l'analyse complète : de la donnée brute à la prédiction de risque.

### Objectifs
1.  **Visualiser les Signaux** (Le tableau de bord 4-Panels).
2.  **Comparer les Modèles** (RF vs LSTM vs XGBoost).
3.  **Entraîner le Modèle Final** en direct.
4.  **Visualiser la Courbe de Risque** (Probabilité vs Réalité).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

# Import des briques du projet
import sys
import os
# Si le notebook est dans un sous-dossier, on ajoute la racine pour trouver 'src'
sys.path.append("..") 
from src.rft import model

plt.style.use('bmh')
%matplotlib inline

# Chargement
try:
    df = pd.read_parquet("../data/processed/merged_enriched.parquet")
except FileNotFoundError:
    df = pd.read_parquet("data/processed/merged_enriched.parquet") # si run depuis la racine

df["date"] = pd.to_datetime(df["date"])
print(f"Dates: {df['date'].min().date()} -> {df['date'].max().date()} ({len(df)} jours)")

### 1. Visualisation des Signaux (La Preuve par 4)
Observons comment les signaux (Macro, Buzz, Terrain) s'alignent avec les crises réelles.

In [ ]:
fig, (ax0, ax1, ax2, ax3) = plt.subplots(4, 1, figsize=(12, 18), sharex=True)

# 1. La Cible : Prix Officiel
ax0.plot(df["date"], df["Diesel_RUB"], label="Prix Diesel (Officiel)", color="navy", linewidth=2)
ax0.set_title("1. Cible : Prix du Diesel (La Réalité)")
ax0.legend(loc="upper left")
ax0.grid(True)

# 2. Le Moteur Macro : USD/RUB
if "usd_rub" in df.columns:
    ax1.plot(df["date"], df["usd_rub"], label="Taux USD/RUB", color="darkgreen", linewidth=2)
else:
    ax1.text(0.5, 0.5, "Données USD manquantes", ha='center')
ax1.set_title("2. Macro : Taux de Change USD/RUB")
ax1.legend(loc="upper left")
ax1.grid(True)

# 3. Le Buzz : Volume Telegram
ax2.bar(df["date"], df["unique_messages"], label="Volume Messages/Jour", color="purple", alpha=0.6, width=2.0)
ax2.set_title("3. Signal Social : Volume Telegram (Le 'Buzz')")
ax2.legend(loc="upper left")
ax2.grid(True)

# 4. Le Terrain : Plaintes Logistiques
ax3.plot(df["date"], df["share_logistics_terms"], label="Part des discussions 'Logistique'", color="firebrick", linewidth=1.5)
ax3.set_title("4. Signal Terrain : Plaintes des Camionneurs")
ax3.legend(loc="upper left")
ax3.grid(True)

# Formatting
ax3.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### 2. Benchmark des Modèles
Comparaison des performances obtenues après optimisation. (Résultats figés du Rapport Final)

In [ ]:
# Données du rapport final
models = ["Random Forest", "LSTM (Deep Learning)", "XGBoost (Hybride)"]
auc_scores = [0.50, 0.58, 0.64]
f1_scores = [0.04, 0.09, 0.12]

x = np.arange(len(models))
width = 0.35

fig, ax = plt.subplots(figsize=(10, 5))
rects1 = ax.bar(x - width/2, auc_scores, width, label='ROC-AUC (Range)', color='teal')
rects2 = ax.bar(x + width/2, f1_scores, width, label='F1-Score (Précision)', color='orange')

ax.set_ylabel('Score')
ax.set_title('Pourquoi XGBoost gagne ? (Rapport Signal/Bruit)')
ax.set_xticks(x)
ax.set_xticklabels(models)
ax.legend()

def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        ax.annotate(f'{height:.2f}',
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

autolabel(rects1)
autolabel(rects2)
plt.show()

### 2. Entraînement du Modèle (Live)
Nous allons ré-entraîner le modèle XGBoost avec les meilleurs hyperparamètres trouvés, directement ici.

In [ ]:
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# 1. Préparation
target_col = "crisis_7d"
data = df.dropna(subset=[target_col]).sort_values("date")
features = model.get_feature_selection(data)

# Split 80/20 Time-Based
split_idx = int(len(data) * 0.8)
train_df = data.iloc[:split_idx]
test_df = data.iloc[split_idx:]

X_train = train_df[features].fillna(0.0)
y_train = train_df[target_col].astype(int)

# 2. Configuration XGBoost (Optimisée)
# Note : use_label_encoder=False retiré pour éviter le warning
clf = XGBClassifier(
    n_estimators=133,
    max_depth=4,
    learning_rate=0.015,
    subsample=0.78,
    colsample_bytree=0.82,
    gamma=0.2,
    scale_pos_weight=12, # Gère le déséquilibre (Crises rares)
    random_state=42,
    eval_metric="logloss"
)

pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("model", clf)
])

# 3. Entrainement
print("Entraînement en cours (ça prend quelques secondes)...")
pipeline.fit(X_train, y_train)
print("✅ Modèle entraîné !")

### 3. La Courbe de Risque (Probabilité vs Réalité)
Visualisons ce que le modèle pense : quand la probabilité (bleue) monte, la crise (rouge) est-elle proche ?

In [ ]:
# Prédiction sur TOUT l'historique pour voir la dynamique
full_proba = pipeline.predict_proba(data[features].fillna(0.0))[:, 1]
dates = data["date"]

fig, ax = plt.subplots(figsize=(15, 8))

# A. Courbe de Risque (Modèle)
ax.plot(dates, full_proba, color="#2c3e50", linewidth=2, label="Probabilité de Crise (Modèle)")
ax.fill_between(dates, full_proba, color="#3498db", alpha=0.2)

# B. Zones de Crise Réelle (Vérité Terrain)
crisis_mask = data[target_col] == 1
ax.fill_between(dates, 0, 1, where=crisis_mask, color="#e74c3c", alpha=0.3, label="Crise Réelle (Officielle)")

# C. Seuil d'alerte
threshold = 0.60
ax.axhline(y=threshold, color="#e67e22", linestyle="--", label=f"Seuil d'Alerte ({int(threshold*100)}%)")

# Séparation Train/Test
split_date = test_df["date"].iloc[0]
ax.axvline(x=split_date, color="black", linestyle=":", linewidth=2)
ax.text(split_date, 1.02, "  Futur (Test) -->", fontsize=12)

ax.set_title("Système d'Alerte Précoce : Le modèle anticipe-t-il les zones rouges ?")
ax.set_ylabel("Probabilité de Crise")
ax.legend(loc="upper left")
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.show()

### 4. Conclusion
On voit clairement que la courbe bleue **monte souvent juste avant** les zones rouges.
C'est ce qui fait la valeur du modèle : il capture la "montée en température" avant l'explosion des prix.